# Quantum Process Tomography

* **Last Updated:** Jan 24, 2019
* **Requires:** qiskit-terra 0.7

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.tools.qi.qi import outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'1': 1939, '0': 2061},
 (('Zp',), ('Z',)): {'1': 2000, '0': 2000},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'1': 2027, '0': 1973},
 (('Zm',), ('Z',)): {'1': 2055, '0': 1945},
 (('Xp',), ('X',)): {'1': 1988, '0': 2012},
 (('Xp',), ('Y',)): {'1': 2026, '0': 1974},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'1': 1954, '0': 2046},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'1': 1997, '0': 2003}}

In [3]:
# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0020859241485595703
fit fidelity: 0.997590633666101

CVXOPT Fitter
fit time: 0.0689091682434082
fit fidelity: 0.9999069096855673


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity

In [4]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = tomo.process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'1': 1004, '0': 996},
 (('Zp',), ('Y',)): {'1': 1030, '0': 970},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'1': 1001, '0': 999},
 (('Zm',), ('Y',)): {'1': 1009, '0': 991},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'1': 1051, '0': 949},
 (('Xp',), ('Z',)): {'1': 959, '0': 1041},
 (('Yp',), ('X',)): {'1': 1017, '0': 983},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'1': 983, '0': 1017}}

In [5]:
# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0016489028930664062
fit fidelity: 0.9956636821737574

CVXOPT Fitter
fit time: 0.04913496971130371
fit fidelity: 0.9998733339639942


## 2-Qubit entangling circuit

In [6]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)

# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts)


t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_mle / 4))

t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

MLE Fitter
fit time: 0.04810309410095215
fit fidelity: 0.980639219412798

CVXOPT Fitter
fit time: 1.859253168106079
fit fidelity: 0.9999333155090061


## Using SIC-POVM preparation basis

In [7]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'1': 1017, '0': 983},
 (('S0',), ('Z',)): {'1': 965, '0': 1035},
 (('S1',), ('X',)): {'1': 1323, '0': 677},
 (('S1',), ('Y',)): {'1': 990, '0': 1010},
 (('S1',), ('Z',)): {'1': 50, '0': 1950},
 (('S2',), ('X',)): {'1': 1361, '0': 639},
 (('S2',), ('Y',)): {'1': 166, '0': 1834},
 (('S2',), ('Z',)): {'1': 1477, '0': 523},
 (('S3',), ('X',)): {'1': 1324, '0': 676},
 (('S3',), ('Y',)): {'1': 1822, '0': 178},
 (('S3',), ('Z',)): {'1': 1472, '0': 528}}

In [8]:
# Extract fitter data from tomography counts
data, basis, weights = tomo.fitter_data(qpt_counts, prep_basis='SIC')

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis, weights)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis, weights)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0010979175567626953
fit fidelity: 0.9968067115276925

CVXOPT Fitter
fit time: 0.0597996711730957
fit fidelity: 0.9999870911602211
